## This script downloads hourly airquality data from DATABC's ftp server and agregates it by month and year

In [1]:
import pandas as pd
import numpy as np
import urllib.request 
import os


Host = "ftp://ftp.env.gov.bc.ca/"

## 2009 is the first year the data is aggregated yearly - The 2021 data is not available yet
Last_Year = 2020
# Years = [str(x) for x in range(2010,Last_Year+1)]
Years = [str(x) for x in range(2010,Last_Year+1)]
## The gas/particulate data available 
Species = ["TEMP_MEAN","PM25","O3","CO"] # Other Species Avaialble - "H2S","NO","NO2","PM10","SO2","TRS"

def ReadFile(Data = None):
    if Data is None:
        Data = pd.read_csv('Temp.csv',parse_dates={'datetime':[0]},index_col=['datetime'],
                              dtype={"DATE_PST":str,"STATION_NAME":str,"EMS_ID":str,"PARAMETER":str,
                                     "INSTRUMENT":str,"RAW_VALUE":float,"UNIT":str,"ROUNDED_VALUE":float},encoding='ISO-8859-1')
        Data['Month']=Data.index.month
        Data['Year']=Data.index.year
    else:
        NewData = pd.read_csv('Temp.csv',parse_dates={'datetime':[0]},index_col=['datetime'],
                              dtype={"DATE_PST":str,"STATION_NAME":str,"EMS_ID":str,"PARAMETER":str,
                                     "INSTRUMENT":str,"RAW_VALUE":float,"UNIT":str,"ROUNDED_VALUE":float},encoding='ISO-8859-1')
        NewData['Month']=NewData.index.month
        NewData['Year']=NewData.index.year
        Data = Data.append(NewData)
    return(Data)

for species in Species:
    print('Downloading '+species)
    Data = None    
    for year in Years:
        print(year)
        Data_Path = "pub/outgoing/AIR/AnnualSummary/"
        path = Host+Data_Path+year+'/'+species+'.csv'
        urllib.request.urlretrieve(path, 'Temp.csv')
        Data = ReadFile(Data)
#     if Last_Year
    
    Data_Path = 'pub/outgoing/AIR/Hourly_Raw_Air_Data/Year_to_Date/'
    path = Host+Data_Path+species+'.csv'
    urllib.request.urlretrieve(path, 'Temp.csv')
    Data = ReadFile(Data)
        
    # Agregate data by year and write to a file
    if species == 'TEMP_MEAN':
        species = 'TEMP'
    
    AggData = Data.groupby(['EMS_ID','Year']).agg({'RAW_VALUE':'mean'})
    Yearly = AggData.unstack()['RAW_VALUE'].to_csv('Data/'+species+'_Yearly_Averages.csv')
    
    Data['Year_Month'] = Data['Year']*100+Data['Month']
    AggData = Data.groupby(['EMS_ID','Year_Month']).agg({'RAW_VALUE':'mean'})
    AggData.unstack()['RAW_VALUE'].to_csv('Data/'+species+'_MonthlyAverages.csv')
    
       
    Summer = Data.loc[((Data['Month']>=6)&(Data['Month']<=9))].copy()
    AggData = Summer.groupby(['EMS_ID','Year']).agg({'RAW_VALUE':'mean'})
    AggData.unstack()['RAW_VALUE'].to_csv('Data/'+species+'_SummerAverages.csv')
    
#     Last_Year = Data.loc[Data.index.year>=2021]

#     Last_Year=Last_Year.groupby([Last_Year.index,Last_Year.EMS_ID]).mean().unstack()['RAW_VALUE']
    
    
    Daily=Data.groupby([Data.index,Data.EMS_ID]).mean().unstack()['RAW_VALUE']

#     print(Daily)

    Daily.loc[Daily.index.year>=2020].to_csv('Data/'+species+'_Daily_Averages_2020_Onward.csv')
    
    print('Completed '+species,': there are ',AggData.unstack().shape[0],' stations available to analyze.')
    print()
os.remove('Temp.csv')
print('Done!!')

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Completed TEMP : there are  140  stations available to analyze.

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Completed PM25 : there are  112  stations available to analyze.

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Completed O3 : there are  74  stations available to analyze.

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Completed CO : there are  36  stations available to analyze.

Done!!


In [ ]:
Sites = pd.read_csv('ftp://ftp.env.gov.bc.ca/pub/outgoing/AIR/Air_Monitoring_Stations/bc_air_monitoring_stations.csv')
Sites.to_csv('MonitoringStations.csv')
Sites

In [ ]:
DF = pd.read_csv('PM25.csv')

DF['STATION_NAME'].unique()

# # Last_Year = Data.loc[Data.index.year>=2021]

# Daily=Data.groupby([Data.index,Data.EMS_ID]).mean().unstack()['RAW_VALUE']

# print(Daily)

In [ ]:
import chardet
with open('Temp.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result